<a href="https://colab.research.google.com/github/LSDtopotools/lsdtt_notebooks/blob/master/lsdtopotools/starting_guide_to_lsdtopotools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Starter's guide to lsdtopotools without condacolab (LEGACY ONLY. NEW USERS SHOULD USE CONDACOLAB)

This is a starter's guide to *lsdtopotools* (https://lsdtopotools.github.io/LSDTT_documentation/) which includes our visualisation tools and our command line tools. 

This document was last updated by Simon M. Mudd on 16/09/2022

**This version does not use condacolab. `condacolab` is now the preferred installation method.**

**Please see the notebook `lsdtopotools_with_condacolab` or any of the other notebooks in this directory.**

This notebook gives a very basic tour of the *lsdtopotools* command line tools. 

**You need to first install lsdtopotools, which is done using something called conda. I'm afraid it takes a few minutes to install when you start this notebook.**

If you use this software for your research, please cite https://doi.org/10.5281/zenodo.3245040

## First install the python tools

We first install lsdviztools. This step takes about 45 seconds. 

In [ ]:
!pip install lsdviztools &> /dev/null

## Now install lsdtt command line tools (will take ~4 minutes total)

First we download miniconda, which takes about 30 seconds. We need this to install other parts of the software

In [ ]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-py37_4.8.2-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.anaconda.com/miniconda/$MINICONDA_INSTALLER_SCRIPT &> /dev/null
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX &> /dev/null

The next step is to set the conda channels, install mamba (because it installs *lsdtopotools* much faster than conda) and then finally lsdtopotools. This takes ~3.5 minutes. 

In [ ]:
!conda config --add channels conda-forge &> /dev/null
!conda config --set channel_priority strict &> /dev/null
!conda install -c conda-forge -y mamba &> /dev/null
!mamba install -c conda-forge -y lsdtopotools &> /dev/null

## Test to see if it worked

Now we perform some tests to see if it all worked. First we try to import some packages for visualisation. 

In [ ]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio
import lsdviztools.lsdmapwrappers as lsdmw

Now we download some data that we will look at using *lsdviztools*. 

In [ ]:
Dataset_prefix = "SB"

# YOU NEED TO PUT YOUR API KEY IN A FILE
your_OT_api_key_file = "my_OT_api_key.txt"

with open(your_OT_api_key_file, 'r') as file:
    print("I am reading you OT API key from the file "+your_OT_api_key_file)
    api_key = file.read().rstrip()
    print("Your api key starts with: "+api_key[0:4])


SB_DEM = bmt.ot_scraper(source = "SRTM30",longitude_W = -120.464655, longitude_E = -120.254214, 
                        latitude_S = 34.440538, latitude_N = 34.610770,
                        prefix = Dataset_prefix,
                        api_key_file = your_OT_api_key_file)
SB_DEM.print_parameters()
SB_DEM.download_pythonic()
DataDirectory = "./"
Fname = Dataset_prefix+"_SRTMGL1.tif"
gio.convert4lsdtt(DataDirectory,Fname)

In [ ]:
lsdtt_parameters = {"write_hillshade" : "true",  
                    "surface_fitting_radius" : "60",
                    "print_slope" : "true"}
r_prefix = Dataset_prefix+"_SRTMGL1_UTM"
w_prefix = Dataset_prefix+"_SRTMGL1_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
%matplotlib inline
Base_file = Dataset_prefix+"_SRTMGL1_UTM"
this_img = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth", save_fig=False, size_format="geomorphology")

If you got an image after the last command, it means that `lsdtopotools` and `lsdviztools` are both working. Congratulations! You are now ready for a basic introduction.

## Now for a basic introduction

Okay, *lsdtopotools* is not actually in python. We have a very simple python interface for it, however, which calls the underlying *lsdtopotools* commands. 

So if it is not python, what is it? Well, *lsdtopotools* is a series of **command line tools**. There are (presently) 6 of them: 

*   `lsdtt-basic-metrics`
*   `lsdtt-chi-mapping`
*   `lsdtt-channel-extraction`
*   `lsdtt-cosmo-tool`
*   `lsdtt-valley-metrics`
*   `lsdtt-hillslope-channel-coupling`


### Wait, what is a command line tool?


You can call a command line tool with the `!` character in a notebook. Here is a very simple command line call, which shows us what files we have sitting here. 

In [ ]:
!ls

Now let's call one of the *lsdtopotools* command line tools. 

In [ ]:
!lsdtt-basic-metrics

The above text means that the system sees `lsdtt-basic-metrics` but it hasn't run any analysis because you need to give it something called a `driver` file. This is basically a list of commands. So what commands can you give it? You can call for help, which spits out a comma separated value file (a bit like a simple excel file) that we can look (I will use `pandas` for this because it is the best way to look at csv files). 

In [ ]:
!lsdtt-basic-metrics -h

In [ ]:
import pandas as pd
pd.__version__
lsdttbm_help = pd.read_csv('lsdtt-basic-metrics-README.csv',error_bad_lines=False)
lsdttbm_help.head(5)

What you see above is just the first few lines of all the stuff you can tell `lsdtt-basic-metrics` to do. The switches to turn on different topographic analyses are the ones where the `type` is a boolean. 

In [ ]:
lsdttbm_switches = lsdttbm_help[lsdttbm_help['type'] == "bool"]
lsdttbm_switches.head(10)

Yes, there are many, many of these in `lsdtt-basic-metrics`. The other tools don't have as many options. Okay, how do I use these? 

If you were a linux native you would write a file to tell `lsdtopotools` what to do. But in the python environment we feed the instructions as a dictionary, like the following:

In [ ]:
lsdtt_parameters = {"write_hillshade" : "true",  
                    "surface_fitting_radius" : "60",
                    "print_slope" : "true"}

**Important** you need to always have quotation marks (even if the value you use is a number).

To make the dictionary you have the name of the parameter (in quotation marks), then a colon, then the value of the parameter (in quotation marks). These are all contained in curly brackets and separated by commas.

This above dictionary is the set of commands used to make the hillshade in the first test. This command also makes a slope map. The dictionary itself doesn't do anything, you need to call the driver 